In [35]:
import cv2
import os
import time
import numpy as np
import matplotlib.pyplot as plt

In [36]:
video_data_dir = '/media/hitsuji/Back/Dance Data/Videos/Advanced/'
frame_data_dir = '/media/hitsuji/Back/Dance Data/Skeleton/Advanced/'

In [37]:
# video_links = np.load('/media/hitsuji/Back/Dance Data/aist_basic_1200.npy', allow_pickle=True).tolist()

In [38]:
video_groups = ['gLH']

In [39]:
# cap = cv2.VideoCapture('/media/hitsuji/Back/Dance Data/Videos/Basic/gBR/1.mp4')

In [40]:
MODE = "MPI"

protoFile = "pose/mpi/pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "pose/mpi/pose_iter_160000.caffemodel"
nPoints = 15
POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13] ]

In [41]:
inWidth = 256
inHeight = 256

net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

In [42]:
def PoseEstimate(frame):
    frameCopy = np.copy(frame)
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    threshold = 0.1
    
    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                          (0, 0, 0), swapRB=False, crop=False)

    net.setInput(inpBlob)

    output = net.forward()
    H = output.shape[2]
    W = output.shape[3]
        
    blank_frame = np.zeros((inHeight, inWidth, 3), np.uint8)
    
    # Empty list to store the detected keypoints
    points = []

    for i in range(nPoints):
        # confidence map of corresponding body's part.
        probMap = output[0, i, :, :]

        # Find global maxima of the probMap.
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        # Scale the point to fit on the original image
        x = (frameWidth * point[0]) / W
        y = (frameHeight * point[1]) / H

        if prob > threshold : 
            # Add the point to the list if the probability is greater than the threshold
            points.append((int(x), int(y)))
        else :
            points.append(None)

    
    # Draw Skeleton
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]

        if points[partA] and points[partB]:
            cv2.line(blank_frame, points[partA], points[partB], (255, 255, 255), 8)

#     cv2.imwrite("./frame_1d.jpg", blank_frame)
    return blank_frame
    
    

In [43]:
x = 650
w = 600
y = 350
h = 600

dsize_in = (256, 256)

In [44]:
for genre in video_groups:
    os.mkdir(frame_data_dir + genre)
    print(genre)
    video_genre_dir = video_data_dir + genre + '/'
    video_files = [name for name in 
                   os.listdir(video_genre_dir) if os.path.isfile(os.path.join(video_genre_dir, name))]
    for video_file in video_files:
        frame_genre_dir = frame_data_dir + genre + '/'
        file_index = video_file.split(".")[0]
        os.mkdir(frame_genre_dir + file_index)
        cap = cv2.VideoCapture(video_genre_dir + video_file)
        success, frame = cap.read()
        count = 0
        while success:
            frame = frame[y:y+h, x:x+w]
            frame = cv2.resize(frame, dsize_in)
            skeleton = PoseEstimate(frame)
            skeleton = cv2.cvtColor(skeleton, cv2.COLOR_BGR2GRAY)
            if count:
                if count % 3 == 0:
                    cv2.imwrite(frame_genre_dir + file_index + "/frame_%d.jpg" % count, skeleton)
            success, frame = cap.read()
            count += 1

gLH


KeyboardInterrupt: 